In [1]:
%matplotlib inline
import numpy as np
import gp_grid.kern
import matplotlib.pyplot as plt
import GPy.kern
from gp_grid.misc import rastrigin

# Standard Kernels
Compare the covariance Matricies with GPy Kernels

In [2]:
for d in [1,10]:
    np.random.seed(0)
    N = 100
    x = np.random.uniform(size=(N,d)) # generate dataset
    y = rastrigin((x*2-1)*5.12)

    # RBF
    np.testing.assert_array_almost_equal(
        gp_grid.kern.RBF(d, lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.RBF(    d, lengthscale=0.5, variance=0.5).K(x))
    
    # Exponential
    np.testing.assert_array_almost_equal(
        gp_grid.kern.Exponential(d, lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.Exponential(    d, lengthscale=0.5, variance=0.5).K(x))
    
    # Matern32
    np.testing.assert_array_almost_equal(
        gp_grid.kern.Matern32(d, lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.Matern32(    d, lengthscale=0.5, variance=0.5).K(x))
    
    # Matern52
    np.testing.assert_array_almost_equal(
        gp_grid.kern.Matern52(d, lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.Matern52(    d, lengthscale=0.5, variance=0.5).K(x))
    
    # GPy - RBF
    np.testing.assert_array_almost_equal(
        gp_grid.kern.GPyKernel(d, kernel="RBF", lengthscale=0.5, variance=0.5).cov(x),
        GPy.kern.RBF(          d, lengthscale=0.5, variance=0.5).K(x))
    
    if d == 1: # these are only for 1d
        # GPy - PeriodicMatern32
        np.testing.assert_array_almost_equal(
            gp_grid.kern.GPyKernel(d, kernel="PeriodicMatern32", lengthscale=0.5, variance=0.5).cov(x),
            GPy.kern.PeriodicMatern32(          d, lengthscale=0.5, variance=0.5).K(x))

# Combining Kernels
Check combining GPy covariance kernels and ensure it works in `gp_grid`. Let the kernel be
$$ k = (k_0 k_1 + k_2)k_3$$

In [3]:
# generate data
np.random.seed(0)
d = 5
N = 100
x = np.random.uniform(size=(N,d)) # generate dataset
y = rastrigin((x*2-1)*5.12)

# get base kernels
kb_gpy = [GPy.kern.RBF(    d, lengthscale=0.5*i+0.5, variance=0.5*i+0.5, name='k%d' % i) 
                  for i in range(4)]
# then combine the base kernels
k_gpy = ((kb_gpy[0] * kb_gpy[1]) + kb_gpy[2]) * kb_gpy[3]

# Wrap the GPy kernel
k_kml = gp_grid.kern.GPyKernel(d, kernel=k_gpy)

# print the kernels
print 'gpy:'
print k_gpy
print 'gp_grid:'
print k_kml

# check to ensure they give the same kernel covariance matrix
np.testing.assert_array_almost_equal( k_kml.cov(x), k_gpy.K(x) )

# test training
m = gp_grid.models.GPRegression(x,y,k_kml)
m.checkgrad()
m.optimize(max_iters=5) # just go for a few iterations to make sure working with getter/setter of params

print "tests completed."

gpy:
  mul.                    |  value  |  constraints  |  priors
  sum.mul.k0.variance     |    0.5  |      +ve      |        
  sum.mul.k0.lengthscale  |    0.5  |      +ve      |        
  sum.mul.k1.variance     |    1.0  |      +ve      |        
  sum.mul.k1.lengthscale  |    1.0  |      +ve      |        
  sum.k2.variance         |    1.5  |      +ve      |        
  sum.k2.lengthscale      |    1.5  |      +ve      |        
  k3.variance             |    2.0  |      +ve      |        
  k3.lengthscale          |    2.0  |      +ve      |        
gp_grid:

GPy - <GPy.kern.src.prod.Prod object at 0x7f030cdb8c10> kernel
| Name        |   Value | Constraint   |
|-------------+---------+--------------|
| variance    |     0.5 | ['+ve']      |
| lengthscale |     0.5 | ['+ve']      |
| variance    |     1   | ['+ve']      |
| lengthscale |     1   | ['+ve']      |
| variance    |     1.5 | ['+ve']      |
| lengthscale |     1.5 | ['+ve']      |
| variance    |     2   | ['+ve']   